<a href="https://colab.research.google.com/github/hongqin/deep_learning_DNA/blob/master/learnseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this tutorial, we'll walk through running a neural network on genome data.

We need upload file 'humanvsran' and 'humanvsran_label' to CoLab

In [10]:
! ls

humanvsran  humanvsran_label  sample_data


Here you will find four files:<br><font color="#417EAE">humanvsran</font>: contain human 10,000 human sequences and 10,000 random sequences generated by RSAT random-seq.<br><font color="#417EAE">humanvsran_label</font>: labels for <font color="#417EAE">humanvsran</font>, where 1 stands for human sequences and 0 stands for random sequences.<br><font color="#417EAE">predict_seq.py</font>: python code for our CNN.<br><font color="#417EAE">learnseq.ipynb</font>: that's me!



Now lets have a look to our data!

In [11]:
!head humanvsran -n 3

CTACTCGGGAGGCTGAGGCAGGAGAATCACTTGAACCAGGGAGTCAGAGGTTGCAGTGAGCCAAGATCGCACCACTGCACTCCAGCCTGGCAACACAGCGAGAGTCTGTCTCCAAAAAAAAAAAAAAAGGCCCAAGAGCATTTCAAACTGTTTGTTGAGAGGTAATCTTATACATTCAGTTTCTTTTGGCTTACCTGAAGAGGTGCTTGGCTATGGTTAACAAACTACGTAAGCTGGTGAAAAAACCTGT
CACCTTATCCAGAGAAGCTTCTTCTTTTAGAAAATCAAGCAAAACAACTAAGCCAAGACATGTTAAAAAAAAAGTTTGAAAAGAAAGCTGTAAGGAAATACAAGGGGAGGGATTGTTAGATATGAGTTCTAAATTTCTTTTCAAAAAATCGATATGTCAGTATGTTCAATTCTTTGCCTTCTACTTTTAAACTTCCTCATAAAGCAACCTTTTTCAATTACCTGCTCCACCCTGACTCATTCTGATCACC
AAAGGGGCTGATAGAAAAATAAAGAGATTTGGCCAGGTACGGTGGCTCACTCCTGTAATCCCAGCACTTTGGGAGGCCGAGGCTGGCGGATCACCTGAGGTCGGGAGTTCGAGACCAGCCTGACCAACATGGAGAACCCCTGTCTCTACTAAAATTACAAAATTAGCCAGGCATGGTGGCACATGTCTGTAATCCCAGCTACTTGGGAGGCTGAGGCAGGAGGATCATTTGAACCAGGGAGGCAGAGGTT


These are sequences randomly cut from human chromosome 1. Now have a look to the label file also.

In [12]:
! tail humanvsran_label -n3

0
0
0


All 1s since they are all human sequence, so if you 'tail' the file, it will be all zeros!

Now lets begin to train our predict_seq program! First import some libraries.

In [13]:
import numpy as np
import os
import tensorflow as tf
import sys
import re
import random
#from tensorflow import keras
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [14]:
#========================= Model ==============================================
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import SGD

Read our sequence and label file.

In [15]:
##Read data
f = open('humanvsran_label', "r")
lines = f.readlines()
f.close()
dat_y = np.zeros((len(lines),2))
for i in range(len(lines)):
	#dat_y[i] = float(dat_y[i].rstrip())
	dat_y[i,int(lines[i].rstrip())] = 1

f = open('humanvsran', "r")
lines = f.readlines()
f.close()

#Convert sequences into one-hot matrix
def DNA_matrix(seq):
	tem2 = ['[aA]','[cC]','[gG]','[tT]']
	for i in range(len(tem2)):
		ind = [m.start() for m in re.finditer(tem2[i], seq)]
		tem = np.zeros(len(seq),dtype=np.int)
		tem[ind] = 1
		if i==0:
			a = np.zeros((len(seq),4))
		a[...,i] = tem
	return a

for i in range(len(lines)):
	tem = lines[i].rstrip()
	if i==0:
		dat_x = np.zeros((len(lines),len(tem),4))
	dat_x[i,] = DNA_matrix(tem)


ind = range(20000)
random.shuffle(list(ind))
x_train=dat_x[ind[0:12000]]
y_train=dat_y[ind[0:12000]]
x_val=dat_x[ind[12001:14001]]
y_val=dat_y[ind[12001:14001]]
x_test=dat_x[14001:20001]
y_test=dat_y[14001:20001]
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [25]:
dat_x.shape

(20000, 250, 4)

In [28]:
dat_x[0].shape

(250, 4)

And you're good to go! Start training!

In [16]:
model=Sequential()
model.add(Conv1D(filters=20,kernel_size=10,strides=1,padding='valid',input_shape=(250,4), activation='relu'))
model.add(MaxPooling1D(pool_size=10, strides=5, padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='loss', patience=2, mode='min')
history = model.fit(x_train, y_train, batch_size=16, epochs=3, verbose=1, validation_data= None,callbacks=[early_stopping])
print(history.history)

Epoch 1/3
750/750 [==============================] - 6s 7ms/step - loss: 0.2500 - accuracy: 0.8956
Epoch 2/3
750/750 [==============================] - 5s 7ms/step - loss: 0.0575 - accuracy: 0.9812
Epoch 3/3
750/750 [==============================] - 5s 7ms/step - loss: 0.0357 - accuracy: 0.9875
{'loss': [0.2500284016132355, 0.05749869719147682, 0.03573708236217499], 'accuracy': [0.8955833315849304, 0.981166660785675, 0.987500011920929]}


Next, evaluate our model

In [22]:
y_score = model.predict(x_test)
loss, acc = model.evaluate(x_test, y_test, verbose=1)
print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

188/188 [==============================] - 1s 4ms/step - loss: 0.0698 - accuracy: 0.9770

Testing loss: 0.0697995126247406, acc: 0.9769961833953857



We can retrive and examine the weight matrix, and convert it into a PWM matrix.

In [20]:
weights = model.layers[0].get_weights()
wt=(np.transpose(weights[0][:,:,0]))
wtm=np.transpose(weights[0][:,:,0]).min(axis=0)
wtp=wt-wtm
wtps=np.sum(wtp, axis=0)
print( np.round((wtp/wtps)*100) )

[[43. 50.  0. 22. 74.  2.  4.  0. 35. 42.]
 [ 0.  0. 76.  0. 26. 45. 95. 31.  0. 25.]
 [13. 14.  4. 50.  0.  0.  1. 57. 31.  0.]
 [45. 36. 20. 28.  0. 53.  0. 12. 35. 33.]]
